In [2]:
import faiss
import numpy as np
from PIL import Image
from clip import clip

/home/aganap12/.local/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
/home/aganap12/.conda/envs/faiss-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
